# Sergio Soler Rocha: Tarea 1

## Ejercicio 1: MapReduce.
Este ejercicio consta de 3 partes, que se detallan en este apartado. Para cada uno de las partes es necesario presentar lo siguiente:

El diseño del programa MapReduce. Este diseño debe contener al menos respuestas a las siguientes preguntas:

1. ¿Cuántos pasos MapReduce son necesarios?

2. ¿Qué hace cada función de cada paso? (No es necesario código ejecutable, una descripción en texto o en pseudocódigo es suficiente).

3. ¿Qué datos se pasan de una función a la siguiente?

Implementación de este diseño. Este código debe ejecutarse en el entorno de desarrollo propuesto para el tema de MapReduce.


In [214]:
! mkdir -p mrjob/join

In [215]:
import os
os.chdir("/media/notebooks/mrjob/join")

In [216]:
! pwd

/media/notebooks/mrjob/join


### Los ficheros countries.csv y clients.csv deben estar descargados en la carpeta /media/notebooks/mrjob/join

In [217]:
cat countries.csv

In [218]:
cat clients.csv

Bertram Pearcy  ,bueno,SO
Steven Ulman  ,regular,ZA
Enid Follansbee  ,malo,GS
Candie Jacko  ,malo,SS
Alana Zufelt  ,regular,ES
Craig Pinkett  ,malo,LK
Carson Levey  ,bueno,GU
Reanna Calabrese  ,regular,GT
Elliott Kosak  ,malo,GG
Yuette Steinman  ,bueno,GN
Grisel Wines  ,regular,GW
Kathryne Dieguez  ,regular,AE
Donna Raabe  ,malo,GB
Norine Mundt  ,bueno,US
Brittaney Amaro  ,bueno,ES
Penni Husted  ,bueno,ES
Delmer Semon  ,malo,IT
Lennie Dunkerson  ,bueno,CA
Mayra Bobb  ,regular,IT
Altagracia Merced  ,regular,CA
Verda Belgrave  ,malo,GB
Jonnie Urban  ,malo,US
Chung Frankum  ,malo,ES
Vincenzo Samples  ,regular,TT
Dominick Barkan  ,bueno,GU
Carisa Ellingwood  ,bueno,TR
Garret Wess  ,regular,TM
Zoraida Muise  ,bueno,GU
Samantha Cusson  ,bueno,PT
Jenine Greenburg  ,regular,PR
Geri Paddock  ,bueno,QA
Antonia Klosterman  ,regular,RE
Moriah Galey  ,malo,RO
Nyla Eckard  ,malo,GB
Arlean Harries  ,malo,US
Kenyatta Lippold  ,malo,ES
Samuel Knipe  ,malo,MV
Jamison Starner  ,malo,ML
Joel Blye  ,regula

In [6]:
%%writefile mrjob-ejercicio.py
#!/usr/bin/env python3

import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, _, line):
    splits = line.rstrip("\n").split(",")

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    else: #  datos de personas
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]

  def reducer(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield key, country[1:] + value[1:]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio.py


Primero ejecutamos el código en local y luego en Hadoop

In [7]:
! python3 mrjob-ejercicio.py /media/notebooks/mrjob/join/countries.csv  \
/media/notebooks/mrjob/join/clients.csv > ouputlocal

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.083115.071059
Running step 1 of 1...
job output is in /tmp/mrjob-ejercicio.root.20231027.083115.071059/output
Streaming final output from /tmp/mrjob-ejercicio.root.20231027.083115.071059/output...
Removing temp directory /tmp/mrjob-ejercicio.root.20231027.083115.071059...


In [8]:
! cat ouputlocal

"MT"	[["Malta", "MT"], ["Joel Blye  ", "regular", "MT"]]
"MV"	[["Maldives", "MV"], ["Samuel Knipe  ", "malo", "MV"]]
"GN"	[["Guinea", "GN"], ["Yuette Steinman  ", "bueno", "GN"]]
"GS"	[["South Georgia and the South Sandwich Islands", "GS"], ["Enid Follansbee  ", "malo", "GS"]]
"GS"	[["South Georgia and the South Sandwich Islands", "GS"], ["Jeni Espinoza  ", "bueno", "GS"]]
"GT"	[["Guatemala", "GT"], ["Reanna Calabrese  ", "regular", "GT"]]
"GU"	[["Guam", "GU"], ["Carson Levey  ", "bueno", "GU"]]
"GU"	[["Guam", "GU"], ["Dominick Barkan  ", "bueno", "GU"]]
"GU"	[["Guam", "GU"], ["Zoraida Muise  ", "bueno", "GU"]]
"GW"	[["Guinea-Bissau", "GW"], ["Grisel Wines  ", "regular", "GW"]]
"AE"	[["United Arab Emirates", "AE"], ["Kathryne Dieguez  ", "regular", "AE"]]
"ZA"	[["South Africa", "ZA"], ["Carolyn Machado  ", "bueno", "ZA"]]
"ZA"	[["South Africa", "ZA"], ["Steven Ulman  ", "regular", "ZA"]]
"IT"	[["Italy", "IT"], ["Delmer Semon  ", "malo", "IT"]]
"IT"	[["Italy", "IT"], ["Mayra Bobb  ", "r

In [177]:
! hdfs dfs -mkdir /tmp/mrjoin
! hdfs dfs -put /media/notebooks/mrjob/join/countries.csv  /tmp/mrjoin
! hdfs dfs -put /media/notebooks/mrjob/join/clients.csv  /tmp/mrjoin

mkdir: `/tmp/mrjoin': File exists
put: `/tmp/mrjoin/countries.csv': File exists
put: `/tmp/mrjoin/clients.csv': File exists


In [10]:
! hdfs dfs -ls  /tmp/mrjoin

Found 2 items
-rw-r--r--   3 root supergroup       1289 2023-10-20 17:00 /tmp/mrjoin/clients.csv
-rw-r--r--   3 root supergroup       4120 2023-10-20 17:00 /tmp/mrjoin/countries.csv


In [11]:
! hdfs dfs -rm /tmp/carpeta/mrjob-join-output/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-join-output

Deleted /tmp/carpeta/mrjob-join-output/_SUCCESS
Deleted /tmp/carpeta/mrjob-join-output/part-00000


In [12]:
! python3 mrjob-ejercicio.py hdfs:///tmp/mrjoin/* -r hadoop --output-dir hdfs:///tmp/carpeta/mrjob-join-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.083140.748518
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.083140.748518/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.083140.748518/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar2660815328813066019/] [] /tmp/streamjob6533803508803016792.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.sta

In [13]:
! hdfs dfs -tail /tmp/carpeta/mrjob-join-output/part-00000

h Galey  ", "malo", "RO"]]
"SB"	[["Solomon Islands", "SB"], ["Anna Rappold  ", "regular", "SB"]]
"SI"	[["Slovenia", "SI"], ["Manda Wingate  ", "regular", "SI"]]
"SO"	[["Somalia", "SO"], ["Albina Lamore  ", "malo", "SO"]]
"SO"	[["Somalia", "SO"], ["Bertram Pearcy  ", "bueno", "SO"]]
"SS"	[["South Sudan", "SS"], ["Candie Jacko  ", "malo", "SS"]]
"SS"	[["South Sudan", "SS"], ["Charisse Salzman  ", "bueno", "SS"]]
"TM"	[["Turkmenistan", "TM"], ["Garret Wess  ", "regular", "TM"]]
"TR"	[["Turkey", "TR"], ["Carisa Ellingwood  ", "bueno", "TR"]]
"TT"	[["Trinidad and Tobago", "TT"], ["Vincenzo Samples  ", "regular", "TT"]]
"US"	[["United States", "US"], ["Arlean Harries  ", "malo", "US"]]
"US"	[["United States", "US"], ["Isis Sorrells  ", "regular", "US"]]
"US"	[["United States", "US"], ["Jonnie Urban  ", "malo", "US"]]
"US"	[["United States", "US"], ["Norine Mundt  ", "bueno", "US"]]
"ZA"	[["South Africa", "ZA"], ["Carolyn Machado  ", "bueno", "ZA"]]
"ZA"	[["South Africa", "ZA"], ["Steven Ulma

### Ejercicio 1.1: Contador de clientes valorados por países.

Partiendo de los ficheros de datos de países y clientes y del código visto en el ejemplo mrjob-join, mejora dicho código para que el programa devuelva cuántos clientes con valoración “bueno” hay en cada país. En concreto, la salida del programa MapReduce debe ser un fichero con el contenido que se muestra en la Figura 1 . Este ejercicio tiene una puntuación de hasta 3 puntos sobre 10.

Del código anterior, primero modificamos la función **mapper_1**, donde añadimos un *elif*, quedándonos solo con los clientes con valoración "bueno". Luego, le añadimos una función **mapper_2** a la cual le pasamos los argumentos de **key** y **value** obtenidos al hacer el join en **reducer_1**. Nuestra nueva **key** va a ser el nombre de los países, y para **value** será el valor numérico 1. Finalmente, añadimos una función **reducer_3**, pasándole la **key** y **value** mencionados anteriormente. En esta función se va a agrupar por su **key**, es decir, por el nombre de los países, y se van a sumar todos los "unos" de **value** para contar cuántos hay de cada.
El siguiente código muestra lo expuesto anteriormente:

In [178]:
%%writefile mrjob-ejercicio.py
#!/usr/bin/env python3

import sys, os, re
from mrjob.job import MRJob
#Importamos la biblioteca
from mrjob.step import MRStep
import statistics


class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper_1(self, _, line):
    splits = line.rstrip("\n").split(",")

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    elif splits[1] == "bueno":   # nos quedamos solo con los clientes "buenos"
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]

  def reducer_1(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield key, country[1:] + value[1:]
        
  def mapper_2(self, key, value):
    country = value[0] # nos quedamos con value[0] que es el nombre del país y su respectiva abreviación
    yield country[0], 1 # con country[0] nos quedamos solo con el nombre del país, el 1 nos servirá como contador
    

  def reducer_2(self, key, value):
    yield key, sum(value) # Agrupa por paises suma todos los "1s" de un mismo país
    
    
        #Definimos los pasos de la aplicación MapReduce.
  def steps(self):
    return [
            MRStep(mapper=self.mapper_1, reducer=self.reducer_1),
            MRStep(mapper=self.mapper_2, reducer=self.reducer_2)
        ]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio.py


In [179]:
# Lo ejecutamos primero en local
! python3 mrjob-ejercicio.py /media/notebooks/mrjob/join/countries.csv  \
/media/notebooks/mrjob/join/clients.csv > ouputlocal 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.132559.053572
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/mrjob-ejercicio.root.20231027.132559.053572/output
Streaming final output from /tmp/mrjob-ejercicio.root.20231027.132559.053572/output...
Removing temp directory /tmp/mrjob-ejercicio.root.20231027.132559.053572...


In [180]:
# Recibimos la cantidad de clientes buenos que hay en cada país
!cat ouputlocal

"South Sudan"	1
"Canada"	1
"Guam"	3
"Spain"	3
"South Georgia and the South Sandwich Islands"	1
"Qatar"	1
"Somalia"	1
"South Africa"	1
"Turkey"	1
"United States"	1
"Guinea"	1
"Portugal"	1


In [181]:
# Borramos la carpeta output y su continido creado anteriormente
! hdfs dfs -rm /tmp/carpeta/mrjob-join-output/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-join-output

Deleted /tmp/carpeta/mrjob-join-output/_SUCCESS
Deleted /tmp/carpeta/mrjob-join-output/part-00000


In [182]:
# Ejecutamos el código en Hadoop
! python3 mrjob-ejercicio.py hdfs:///tmp/mrjoin/* -r hadoop --output-dir hdfs:///tmp/carpeta/mrjob-join-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.132605.376612
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.132605.376612/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.132605.376612/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar2347365816563646201/] [] /tmp/streamjob2343380060419349423.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.sta

In [183]:
# Comprobamos que se ha creado el archivo part-
! hdfs dfs -ls /tmp/carpeta/mrjob-join-output

Found 2 items
-rw-r--r--   3 root supergroup          0 2023-10-27 15:26 /tmp/carpeta/mrjob-join-output/_SUCCESS
-rw-r--r--   3 root supergroup        187 2023-10-27 15:26 /tmp/carpeta/mrjob-join-output/part-00000


In [184]:
# Ejecutamos rl archivo part-00000 mostrandonos a los diferentes paises con el úmero de clientes buenos que disponen
! hdfs dfs -cat /tmp/carpeta/mrjob-join-output/part-00000

"Canada"	1
"Guam"	3
"Guinea"	1
"Portugal"	1
"Qatar"	1
"Somalia"	1
"South Africa"	1
"South Georgia and the South Sandwich Islands"	1
"South Sudan"	1
"Spain"	3
"Turkey"	1
"United States"	1


### Ejercicio 1.2: País con mejores clientes.
Partiendo del código implementado en el ejercicio anterior, extiéndelo para que devuelva el país en el que hay más clientes valorados como “bueno”. En el caso de que haya más de un país con el mismo número de clientes buenos empatados en el primer lugar, se devolverá solamente uno de ellos.

Al código anterior, le añadimos una nueva función **mapper_3**. En este caso, como key ponemos *None* ya que no vamos a hacer ninguna agrupación en el siguiente paso. Como **value**, juntamos en una tupla el número de países con clientes "bueno" y el nombre del país. En el siguiente paso, creamos una función **reducer_3** que nos retorna el valor máximo de **value**, que es la tupla creada en el paso anterior. Al usar **max(value)**, solo nos devuelve un valor de la tupla.

In [185]:
%%writefile mrjob-ejercicio.py
#!/usr/bin/env python3

import sys, os, re
from mrjob.job import MRJob
#Importamos la biblioteca
from mrjob.step import MRStep
import statistics


class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper_1(self, _, line):
    splits = line.rstrip("\n").split(",")

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    elif splits[1] == "bueno":   # nos quedamos solo con los clientes "bueno"
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]

  def reducer_1(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield key, country[1:] + value[1:]
        
  def mapper_2(self, key, value):
    country = value[0] # nos quedamos con value[0] que es el nombre del país y su respectiva abreviación
    yield country[0], 1 # con country[0] nos quedamos solo con el nombre del país, el 1 nos servirá como contador
    

  def reducer_2(self, key, value):
    yield key, sum(value) # Agrupa por paises suma todos los "1s" de un mismo país
    
  def mapper_3(self, key, value):
    yield None, (value, key) # Agrupamos en una tupla como value los valores value y key obtenidos en el paso anterior
    
  def reducer_3(self, _, value):
    yield max(value) # Nos devuelve el valor máximo que encuentra
    
    
        #Definimos los pasos de la aplicación MapReduce.
  def steps(self):
    return [
            MRStep(mapper=self.mapper_1, reducer=self.reducer_1),
            MRStep(mapper=self.mapper_2, reducer=self.reducer_2),
            MRStep(mapper=self.mapper_3, reducer=self.reducer_3)
        ]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio.py


In [186]:
# Ejecutamos primero en Local
! python3 mrjob-ejercicio.py /media/notebooks/mrjob/join/countries.csv  \
/media/notebooks/mrjob/join/clients.csv > ouputlocal 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.132723.371662
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
job output is in /tmp/mrjob-ejercicio.root.20231027.132723.371662/output
Streaming final output from /tmp/mrjob-ejercicio.root.20231027.132723.371662/output...
Removing temp directory /tmp/mrjob-ejercicio.root.20231027.132723.371662...


In [187]:
# Nos devuelve el valor máximo junto con su país
!cat ouputlocal

3	"Spain"


In [188]:
# Borramos la carpeta output y su continido creado anteriormente
! hdfs dfs -rm /tmp/carpeta/mrjob-join-output/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-join-output

Deleted /tmp/carpeta/mrjob-join-output/_SUCCESS
Deleted /tmp/carpeta/mrjob-join-output/part-00000


In [189]:
# Ejecutamos el código en Hadoop
! python3 mrjob-ejercicio.py hdfs:///tmp/mrjoin/* -r hadoop --output-dir hdfs:///tmp/carpeta/mrjob-join-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.132732.046955
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.132732.046955/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.132732.046955/files/
Running step 1 of 3...
  packageJobJar: [/tmp/hadoop-unjar1996947564141434612/] [] /tmp/streamjob6135364981074598771.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.sta

Removing temp directory /tmp/mrjob-ejercicio.root.20231027.132732.046955...


In [190]:
# Comprobamos que se ha creado el archivo part-
! hdfs dfs -ls /tmp/carpeta/mrjob-join-output

Found 2 items
-rw-r--r--   3 root supergroup          0 2023-10-27 15:28 /tmp/carpeta/mrjob-join-output/_SUCCESS
-rw-r--r--   3 root supergroup         10 2023-10-27 15:28 /tmp/carpeta/mrjob-join-output/part-00000


In [191]:
# Ejecutamos rl archivo part-00000 mostrandonos el país con el número máximos de clientes
! hdfs dfs -cat /tmp/carpeta/mrjob-join-output/part-00000

3	"Spain"


### Ejercicio 1.3: Mejorando el país con mejores clientes.
Partiendo del código implementado para el ejercicio anterior, mejóralo para que, en el caso de que haya más de un país empatado con el mayor número de buenos clientes, se devuelvan todos esos países. Utilizando los ficheros de datos sobre países y clientes vistos anteriormente.

El código anterior devuelve únicamente un país y no tiene en cuenta si existen varios países con la misma cantidad de clientes. Para lograr que muestre todos los países en caso de empate, hemos realizado modificaciones en la función **reducer_3** del código previo. Es importante tener en cuenta que **value** es un generador, por lo tanto, una vez que se utiliza un valor, se elimina automáticamente. Para abordar este problema, hemos creado una lista que almacena todos los valores de **value**, permitiéndonos manipularlos con los valores guardados en ella. A continuación, utilizamos la función **max()** para calcular el valor máximo de la lista creada y, posteriormente, mediante un ciclo **for**, devolvemos los países cuyo valor coincida con el máximo.

In [345]:
%%writefile mrjob-ejercicio.py
#!/usr/bin/env python3

import sys, os, re
from mrjob.job import MRJob
#Importamos la biblioteca
from mrjob.step import MRStep
import statistics


class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper_1(self, _, line):
    splits = line.rstrip("\n").split(",")

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    elif splits[1] == "bueno":   # nos quedamos solo con los clientes "buenos"
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]

  def reducer_1(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield key, country[1:] + value[1:]
        
  def mapper_2(self, key, value):
    country = value[0] # nos quedamos con value[0] que es el nombre del país y su respectiva abreviación
    yield country[0], 1 # con country[0] nos quedamos solo con el nombre del país, el 1 nos servirá como contador
    

  def reducer_2(self, key, value):
    yield key, sum(value) # Agrupa por paises suma todos los "1s" de un mismo país
    
  def mapper_3(self, key, value):
    yield None, (value, key) # Agrupamos en una tupla como value los valores value y key obtenidos en el paso anterior
    
  def reducer_3(self, _, values):
    list_values = [] # creamos una lista vacia para introducir los valores de value
    for value in values:
        list_values.append(value) # introducimos los valores 
        
    max_count = max(list_values)[0] # calculamos el el valor máximo
    
    for count, country in list_values:
        if count == max_count: # si el valor máximo es igual a uno de los valores de los países devolvemos el valor y el nombre del país
            yield count, country
    
    
        #Definimos los pasos de la aplicación MapReduce.
  def steps(self):
    return [
            MRStep(mapper=self.mapper_1, reducer=self.reducer_1),
            MRStep(mapper=self.mapper_2, reducer=self.reducer_2),
            MRStep(mapper=self.mapper_3, reducer=self.reducer_3)
        ]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio.py


In [346]:
! python3 mrjob-ejercicio.py /media/notebooks/mrjob/join/countries.csv  \
/media/notebooks/mrjob/join/clients.csv > ouputlocal 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.152925.865679
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
job output is in /tmp/mrjob-ejercicio.root.20231027.152925.865679/output
Streaming final output from /tmp/mrjob-ejercicio.root.20231027.152925.865679/output...
Removing temp directory /tmp/mrjob-ejercicio.root.20231027.152925.865679...


In [347]:
!cat ouputlocal

3	"Guam"
3	"Spain"


In [348]:
# Borramos la carpeta output y su continido creado anteriormente
! hdfs dfs -rm /tmp/carpeta/mrjob-join-output/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-join-output

Deleted /tmp/carpeta/mrjob-join-output/_SUCCESS
Deleted /tmp/carpeta/mrjob-join-output/part-00000


In [349]:
# Ejecutamos el código en Hadoop
! python3 mrjob-ejercicio.py hdfs:///tmp/mrjoin/* -r hadoop --output-dir hdfs:///tmp/carpeta/mrjob-join-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/mrjob-ejercicio.root.20231027.152930.462681
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.152930.462681/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20231027.152930.462681/files/
Running step 1 of 3...
  packageJobJar: [/tmp/hadoop-unjar6221291369973495803/] [] /tmp/streamjob2580921907974502392.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Connecting to ResourceManager at yarnmaster/172.19.0.3:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.sta

Removing temp directory /tmp/mrjob-ejercicio.root.20231027.152930.462681...


In [350]:
# Comprobamos que se ha creado el archivo part-
! hdfs dfs -ls /tmp/carpeta/mrjob-join-output

Found 2 items
-rw-r--r--   3 root supergroup          0 2023-10-27 17:30 /tmp/carpeta/mrjob-join-output/_SUCCESS
-rw-r--r--   3 root supergroup         19 2023-10-27 17:30 /tmp/carpeta/mrjob-join-output/part-00000


In [351]:
# Ejecutamos rl archivo part-00000 mostrandonos el país con el número máximos de clientes
! hdfs dfs -cat /tmp/carpeta/mrjob-join-output/part-00000

3	"Guam"
3	"Spain"


## Ejercicio 2: Hive
En este trabajo se van a utilizar ficheros (disponibles en el curso virtual) descargados de la web del Banco Mundial que contienen información sobre tasa de finalización de la educación de nivel primario en mujeres por cada país del mundo (lo llamaremos “tasa” en adelante) y sobre grupo de ingresos al que pertenece cada país. Son los archivos siguientes:

API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv. Cuya estructura es la siguiente:
- Country Name: Nombre del país.
- Country Code: Código identificador del país.
- Indicator Name: Nombre del indicador que se muestra en el fichero (la tasa mencionada arriba).
- Indicator Code: Código identificador del indicador.
- 1960, 1961… : Una columna por cada año que contiene el valor del indicador para dicho año.

Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv. Cuya estructura es:
- Country Name: Nombre del país.
- Country Code: Código identificador del país.
- Región a la que pertenece el país.
- Income group: Grupo de ingresos al que pertenece el país (ej. "Ingreso mediano alto", "Países de ingreso bajo"…).

In [413]:
# Importamos los ficheros de datos a HDFS con el siguiente comando.
! hdfs dfs -put /media/notebooks/ficheros_tarea2/API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv  /user/root

In [414]:
# Importamos los ficheros de datos a HDFS con el siguiente comando.
! hdfs dfs -put /media/notebooks/ficheros_tarea2/Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv  /user/root

In [355]:
! hdfs dfs -ls /user/root

Found 6 items
-rw-r--r--   3 root supergroup     161733 2023-10-27 18:01 /user/root/API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv
-rw-r--r--   3 root supergroup      17506 2023-10-27 18:01 /user/root/Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv
drwxr-xr-x   - root supergroup          0 2023-10-19 11:07 /user/root/input
drwxrwxrwx   - root supergroup          0 2023-10-23 12:26 /user/root/life_exp
drwxr-xr-x   - root supergroup          0 2023-10-20 15:59 /user/root/output
drwxr-xr-x   - root supergroup          0 2023-10-19 15:39 /user/root/tmp


### Creación de la base de datos
Creamos una base de datos con el nombre de tarea1.2 con el siguiente comando:

In [356]:
#La base de datos se crea con el comando "CREATE DATABASE"
!beeline -u "jdbc:hive2://localhost:10000/" -e "CREATE DATABASE IF NOT EXISTS tarea1 COMMENT 'tarea1.2' WITH DBPROPERTIES ('Creada por' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027180335_ab11ddac-0b63-417b-b659-04a41a5d2247): CREATE DATABASE IF NOT EXISTS tarea1 COMMENT 'tarea1.2' WITH DBPROPERTIES ('Creada por' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231027180335_ab11ddac-0b63-417b-b659-04a41a5d2247); Time taken: 0.59 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027180335_ab11ddac-0b63-417b-b659-04a41a5d2247): CREATE DATABASE IF NOT EXISTS tarea1 COMMENT 'tarea1.2' WITH DBPROPERTIES ('Creada por' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  :

### Creacion de las tablas e importación de los datos
La primera tabla que vamos a crear con el nombre de "tasa" va a ser con los datos de API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv, cuyo archivo ha sido modificado previamente (antes de importarlo a Hadoop) prescindiendo de las dobles comas en los diferentes campos y eliminando la primera linea, la cual hacía referencia a los nombres de las diferentes columnas.

Debido a que contine una columna por cada año desde 1960 hasta 2022, en la siguiente celda vamos a crear un ciclo for que cree todos esos años y los añadimos junto a las columnas fijas: 

In [358]:
# Definimos las columnas fijas
fixed_columns = ["Country_Name STRING", "Country_Code STRING", "Indicator_Name STRING", "Indicator_Code STRING"]

# Definimos el rango de años (1960 a 2021)
start_year = 1960
end_year = 2022

# Creamos una lista de las columnas de años
year_columns = [f"year_{str(year)} FLOAT" for year in range(start_year, end_year + 1)]

# Combinar todas las columnas
all_columns = fixed_columns + year_columns

# Creamos la sentencia Hive
hive_statement = f"CREATE TABLE IF NOT EXISTS tasa ({', '.join(all_columns)})"

print(hive_statement)

CREATE TABLE IF NOT EXISTS tasa (Country_Name STRING, Country_Code STRING, Indicator_Name STRING, Indicator_Code STRING, year_1960 FLOAT, year_1961 FLOAT, year_1962 FLOAT, year_1963 FLOAT, year_1964 FLOAT, year_1965 FLOAT, year_1966 FLOAT, year_1967 FLOAT, year_1968 FLOAT, year_1969 FLOAT, year_1970 FLOAT, year_1971 FLOAT, year_1972 FLOAT, year_1973 FLOAT, year_1974 FLOAT, year_1975 FLOAT, year_1976 FLOAT, year_1977 FLOAT, year_1978 FLOAT, year_1979 FLOAT, year_1980 FLOAT, year_1981 FLOAT, year_1982 FLOAT, year_1983 FLOAT, year_1984 FLOAT, year_1985 FLOAT, year_1986 FLOAT, year_1987 FLOAT, year_1988 FLOAT, year_1989 FLOAT, year_1990 FLOAT, year_1991 FLOAT, year_1992 FLOAT, year_1993 FLOAT, year_1994 FLOAT, year_1995 FLOAT, year_1996 FLOAT, year_1997 FLOAT, year_1998 FLOAT, year_1999 FLOAT, year_2000 FLOAT, year_2001 FLOAT, year_2002 FLOAT, year_2003 FLOAT, year_2004 FLOAT, year_2005 FLOAT, year_2006 FLOAT, year_2007 FLOAT, year_2008 FLOAT, year_2009 FLOAT, year_2010 FLOAT, year_2011 FL

In [409]:
# Copiamos y pegamos CREATE TABLE... obtenido anteriormente en la primera linea y creamos la tabla tasa
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
CREATE TABLE IF NOT EXISTS tasa (Country_Name STRING, Country_Code STRING, Indicator_Name STRING, Indicator_Code STRING, year_1960 FLOAT, year_1961 FLOAT, year_1962 FLOAT, year_1963 FLOAT, year_1964 FLOAT, year_1965 FLOAT, year_1966 FLOAT, year_1967 FLOAT, year_1968 FLOAT, year_1969 FLOAT, year_1970 FLOAT, year_1971 FLOAT, year_1972 FLOAT, year_1973 FLOAT, year_1974 FLOAT, year_1975 FLOAT, year_1976 FLOAT, year_1977 FLOAT, year_1978 FLOAT, year_1979 FLOAT, year_1980 FLOAT, year_1981 FLOAT, year_1982 FLOAT, year_1983 FLOAT, year_1984 FLOAT, year_1985 FLOAT, year_1986 FLOAT, year_1987 FLOAT, year_1988 FLOAT, year_1989 FLOAT, year_1990 FLOAT, year_1991 FLOAT, year_1992 FLOAT, year_1993 FLOAT, year_1994 FLOAT, year_1995 FLOAT, year_1996 FLOAT, year_1997 FLOAT, year_1998 FLOAT, year_1999 FLOAT, year_2000 FLOAT, year_2001 FLOAT, year_2002 FLOAT, year_2003 FLOAT, year_2004 FLOAT, year_2005 FLOAT, year_2006 FLOAT, year_2007 FLOAT, year_2008 FLOAT, year_2009 FLOAT, year_2010 FLOAT, year_2011 FLOAT, year_2012 FLOAT, year_2013 FLOAT, year_2014 FLOAT, year_2015 FLOAT, year_2016 FLOAT, year_2017 FLOAT, year_2018 FLOAT, year_2019 FLOAT, year_2020 FLOAT, year_2021 FLOAT, year_2022 FLOAT) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027195835_e4b5a780-cae4-4ce3-a103-ad46cca3afb4): CREATE TABLE IF NOT EXISTS tasa (Country_Name STRING, Country_Code STRING, Indicator_Name STRING, Indicator_Code STRING, year_1960 FLOAT, year_1961 FLOAT, year_1962 FLOAT, year_1963 FLOAT, year_1964 FLOAT, year_1965 FLOAT, year_1966 FLOAT, year_1967 FLOAT, year_1968 FLOAT, year_1969 FLOAT, year_1970 FLOAT, year_1971 FLOAT, year_1972 FLOAT, year_1973 FLOAT, year_1974 FLOAT, year_1975 FLOAT, year_1976 FLOAT, year_1977 FLOAT, year_1978 FLOAT, year_1979 FLOAT, year_1980 FLOAT, year_1981 FLOAT, year_1982 FLOAT, year_1983 FLOAT, year_1984 FLOAT, year_1985 FLOAT, year_1986 FLOAT, year_1987 FLOAT, year_1988 FLOAT, year_1989 FLOAT, year_1990 FLOAT, year_1991 FLOAT, year_1992 FLOAT, year_1993 FLOAT, year_1994 FLOAT, year_1995 FLOA

In [410]:
# Muestra las tablas existente en la base de datos
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "SHOW \
TABLES"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027195841_87614e9f-d01c-40f9-9812-8a4b13b419dd): SHOW  TABLES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027195841_87614e9f-d01c-40f9-9812-8a4b13b419dd); Time taken: 0.028 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027195841_87614e9f-d01c-40f9-9812-8a4b13b419dd): SHOW  TABLES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20231027195841_87614e9f-d01c-40f9-9812-8a4b13b419dd); Time

In [415]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
LOAD DATA INPATH '/user/root/API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv' INTO TABLE tasa;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027195945_6c43806f-b84a-4e1f-ae9b-8665ec885a6e): LOAD DATA INPATH '/user/root/API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv' INTO TABLE tasa
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231027195945_6c43806f-b84a-4e1f-ae9b-8665ec885a6e); Time taken: 0.035 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027195945_6c43806f-b84a-4e1f-ae9b-8665ec885a6e): LOAD DATA INPATH '/user/root/API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv' INTO TABLE tasa
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : 

Vemos que la tabla ha sido creada con éxito, vamos ahora a hacer una pequeña consulta para ver que los datos están en la tabla de manera coherente:

In [416]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT country_Name, country_Code, indicator_Name, indicator_Code, year_2020 FROM tasa \
LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027195952_a5533ef0-25a1-4920-9533-1b6ac710814b): SELECT country_Name, country_Code, indicator_Name, indicator_Code, year_2020 FROM tasa  LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country_name, type:string, comment:null), FieldSchema(name:country_code, type:string, comment:null), FieldSchema(name:indicator_name, type:string, comment:null), FieldSchema(name:indicator_code, type:string, comment:null), FieldSchema(name:year_2020, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027195952_a5533ef0-25a1-4920-9533-1b6ac710814b); Time taken: 0.084 seconds
INFO  : Concurrency 

Vamos ahora a proceder a crear la siguiente tabla con el nombre de income_group con los datos de Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv

In [417]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
CREATE TABLE IF NOT EXISTS income_group (Country_Name STRING, Country_Code STRING, Region STRING, Income_Group STRING) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200134_1a82dffe-faaa-4cbc-a814-bbdcc31598fb): CREATE TABLE IF NOT EXISTS income_group (Country_Name STRING, Country_Code STRING, Region STRING, Income_Group STRING)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231027200134_1a82dffe-faaa-4cbc-a814-bbdcc31598fb); Time taken: 0.035 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027200134_1a82dffe-faaa-4cbc-a814-bbdcc31598fb): CREATE TABLE IF NOT E

In [418]:
# Muestra las tablas
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "SHOW \
TABLES"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200137_cc06e85d-ca45-4fe6-8428-7d5707650595): SHOW  TABLES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027200137_cc06e85d-ca45-4fe6-8428-7d5707650595); Time taken: 0.027 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027200137_cc06e85d-ca45-4fe6-8428-7d5707650595): SHOW  TABLES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20231027200137_cc06e85d-ca45-4fe6-8428-7d5707650595); Time

In [419]:
## Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
LOAD DATA INPATH '/user/root/Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv' INTO TABLE income_group;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200140_71304330-9719-4f46-b325-afa7b7967e43): LOAD DATA INPATH '/user/root/Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv' INTO TABLE income_group
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231027200140_71304330-9719-4f46-b325-afa7b7967e43); Time taken: 0.041 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027200140_71304330-9719-4f46-b325-afa7b7967e43): LOAD DATA INPATH '/user/root/Metadata_Country_API_SE.PRM.CMPT.FE.ZS_DS2_es_csv_v2_5641106.csv' INTO TABLE income_group
INFO  : St

Hacemos una consulta para verificar que los datos se han introducido correctamente

In [420]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT * FROM income_group \
LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200148_72882ab6-12d3-4e6a-9368-906fd3a47262): SELECT * FROM income_group  LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:income_group.country_name, type:string, comment:null), FieldSchema(name:income_group.country_code, type:string, comment:null), FieldSchema(name:income_group.region, type:string, comment:null), FieldSchema(name:income_group.income_group, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027200148_72882ab6-12d3-4e6a-9368-906fd3a47262); Time taken: 0.083 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(q

### Creación de la Vista
Esta vista tendrá para cada país, su nombre, código, tasa en el año 2018 y su grupo de ingresos.
Hacemos un INNER JOIN de ambas tablas uniéndolas por nombres de paises y por código de paises ya que faltan algunos nombres de paises.

In [421]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
CREATE VIEW tasa_2018 AS \
SELECT tasa.country_code, tasa.country_name, tasa.year_2018, income_group.income_group FROM tasa \
INNER JOIN income_group \
ON (income_group.country_code = tasa.country_code);"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200206_a1a78ac9-86d9-45f7-b204-711c6816ddf5): CREATE VIEW tasa_2018 AS  SELECT tasa.country_code, tasa.country_name, tasa.year_2018, income_group.income_group FROM tasa  INNER JOIN income_group  ON (income_group.country_code = tasa.country_code)
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country_code, type:string, comment:null), FieldSchema(name:country_name, type:string, comment:null), FieldSchema(name:year_2018, type:float, comment:null), FieldSchema(name:income_group, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027200206_a1a78ac9-86d9-45f7-b204-711c6816ddf5); Time 

Hacemos una consulta general para ver que se ha creado correctamente la vista

In [422]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT * FROM tasa_2018 \
LIMIT 10 \
;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200305_36e0d4cc-dec5-4a23-b1c7-17afb824f876): SELECT * FROM tasa_2018  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tasa_2018.country_code, type:string, comment:null), FieldSchema(name:tasa_2018.country_name, type:string, comment:null), FieldSchema(name:tasa_2018.year_2018, type:float, comment:null), FieldSchema(name:tasa_2018.income_group, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027200305_36e0d4cc-dec5-4a23-b1c7-17afb824f876); Time taken: 0.155 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_

Comprobamos también que coincidan el numero de filas en las dos tablas creadas y en la vista:

In [423]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT COUNT(country_code) FROM tasa;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200757_71ac03f6-c408-48b6-9e28-502339dd08fa): SELECT COUNT(country_code) FROM tasa
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:_c0, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027200757_71ac03f6-c408-48b6-9e28-502339dd08fa); Time taken: 0.1 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027200757_71ac03f6-c408-48b6-9e28-502339dd08fa): SELECT COUNT(country_code) FROM tasa
WARN  : Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a different execution e

In [424]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT COUNT(country_code) FROM income_group;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027200838_5b9cc8ec-ec1d-45e3-a39c-f9c5c0261614): SELECT COUNT(country_code) FROM income_group
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:_c0, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027200838_5b9cc8ec-ec1d-45e3-a39c-f9c5c0261614); Time taken: 0.09 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027200838_5b9cc8ec-ec1d-45e3-a39c-f9c5c0261614): SELECT COUNT(country_code) FROM income_group
WARN  : Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a diff

In [425]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT COUNT(country_code) FROM tasa_2018;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231027204414_8a8ea5b0-fb2c-4398-95f4-9421c1d1601e): SELECT COUNT(country_code) FROM tasa_2018
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:_c0, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231027204414_8a8ea5b0-fb2c-4398-95f4-9421c1d1601e); Time taken: 0.172 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231027204414_8a8ea5b0-fb2c-4398-95f4-9421c1d1601e): SELECT COUNT(country_code) FROM tasa_2018
WARN  : Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a different

### ¿Cuál es la tasa en el año 2018 en España?
Para calcular la tasa de España vamos a usar la vista creada anteriormente

In [426]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT country_name, year_2018 FROM tasa_2018 \
WHERE country_name = 'España';"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028103033_fb27a373-0db8-4876-a535-04811526957e): SELECT country_name, year_2018 FROM tasa_2018  WHERE country_name = 'Espa?a'
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country_name, type:string, comment:null), FieldSchema(name:year_2018, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028103033_fb27a373-0db8-4876-a535-04811526957e); Time taken: 0.167 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028103033_fb27a373-0db8-4876-a535-04811526957e): SELECT country_name, year_2018 FROM tasa_2018  WHERE country_

### ¿Cuál es la media de la tasa en 2018 para los países del grupo de “Países de ingreso bajo”?

Para llegar al resultado final vamos a usar dos métodos diferentes:
-  El primero de ellos consistirá en agrupar por los diferentes income_group calcular sus respectivos promedios de tasa del año 2018 y finalmente mostramos sólo los income_group de "Países de ingreso bajo".

- El segundo método consiste en sólo hacer el promedio con los paises cuyo income_group coincida con el de "Países de ingreso bajo".

In [429]:
# Primer método
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT income_group, AVG(year_2018) as media_paises_2018 \
FROM tasa_2018 \
GROUP BY income_group \
HAVING income_group = 'Países de ingreso bajo';"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028104658_1586054e-2483-49f5-8045-760e2fa55994): SELECT income_group, AVG(year_2018) as media_paises_2018  FROM tasa_2018  GROUP BY income_group  HAVING income_group = 'Pa?ses de ingreso bajo'
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:income_group, type:string, comment:null), FieldSchema(name:media_paises_2018, type:double, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028104658_1586054e-2483-49f5-8045-760e2fa55994); Time taken: 0.222 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028104658_1586054e-2483-49f5-8045-

In [430]:
# Segundo método
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT AVG(year_2018) as media_paises_ingreso_bajo_2018 FROM tasa_2018\
WHERE income_group = 'Países de ingreso bajo';"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028104753_e6a1f7b4-484a-405e-bd6d-b2c03b75884f): SELECT AVG(year_2018) as media_paises_ingreso_bajo_2018 FROM tasa_2018 WHERE income_group = 'Pa?ses de ingreso bajo'
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:media_paises_ingreso_bajo_2018, type:double, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028104753_e6a1f7b4-484a-405e-bd6d-b2c03b75884f); Time taken: 0.21 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028104753_e6a1f7b4-484a-405e-bd6d-b2c03b75884f): SELECT AVG(year_2018) as media_paises_ingreso_bajo_2018 FRO

In [2]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT country_name, year_2020 FROM tasa \
WHERE year_2020 = (SELECT MAX(year_2020) FROM tasa);"


Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231026105121_d7273d33-0963-4b61-836c-5f6caf112ee0): SELECT country_name, year_2020 FROM tasa  WHERE year_2020 = (SELECT MAX(year_2020) FROM tasa)
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country_name, type:string, comment:null), FieldSchema(name:year_2020, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231026105121_d7273d33-0963-4b61-836c-5f6caf112ee0); Time taken: 0.154 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231026105121_d7273d33-0963-4b61-836c-5f6caf112ee0): SELECT country_name, year_2020 FROM tasa  WH

### ¿Cuáles son los cinco países con mayor tasa en 2020?
En la tabla tasa mostramos el nombre de los países y su respectiva tasa en el 2020, ordenamos de manera descendente por por la tasa de 2020 y mostramos sólo los 5 primeros resultados

In [5]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT country_name, year_2020 FROM tasa \
ORDER BY year_2020 DESC \
LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231026105437_922da533-2116-4657-bab2-d38911bd72f8): SELECT country_name, year_2020 FROM tasa  ORDER BY year_2020 DESC  LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country_name, type:string, comment:null), FieldSchema(name:year_2020, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231026105437_922da533-2116-4657-bab2-d38911bd72f8); Time taken: 0.085 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231026105437_922da533-2116-4657-bab2-d38911bd72f8): SELECT country_name, year_2020 FROM tasa  ORDER BY year_2020 DE

### ¿Cuántos países de la región “Oriente Medio y Norte de África (excluido altos ingresos)” pertenecen a cada grupo de ingresos?
Seleccionamos solo los de la región de "Oriente Medio y Norte de África (excluido altos ingresos)" agrupamos por income_group y contamos el número de países por cada grupo

In [432]:
!beeline -u "jdbc:hive2://localhost:10000/tarea1" -e "\
SELECT income_group, COUNT(*) as number_of_countries FROM income_group \
WHERE region = 'Oriente Medio y Norte de África (excluido altos ingresos)'\
GROUP BY income_group;"

Connecting to jdbc:hive2://localhost:10000/tarea1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028112257_db14fccf-e273-4672-a776-a9bac1335240): SELECT income_group, COUNT(*) as number_of_countries FROM income_group  WHERE region = 'Oriente Medio y Norte de ?frica (excluido altos ingresos)' GROUP BY income_group
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:income_group, type:string, comment:null), FieldSchema(name:number_of_countries, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028112257_db14fccf-e273-4672-a776-a9bac1335240); Time taken: 0.191 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_202310281122

### Selecciona un dataset, impórtalo en las tablas de Hive necesarias e implementa al menos dos consultas sobre dicho dataset.

Vamos a seleccionar el dataset *Football Data from Transfermarkt* de https://www.kaggle.com/datasets/davidcariboo/player-scores/data?select=players.csv. El dataset contiene diferentes archivos csv sobre datos de competiciones de fúbol a nivel profesional. En la siguiente imagen se muestra la relación entre las diferentes tablas del dataset:

![relación de tablas](diagram.svg)

Nuestro propósito en este ejercicio es mostrar algunas estadísticas de jugadores y clubes por lo que vamos a usar las siguientes tablas:

- competitions: contiene información sobre las diferentes competiciones futbolísticas.
- games: información sobre las partidos jugados.
- clubs: información de cada club.
- club_games: información sobre el papel de cada club en los diferentes partidos.
- players: información individual de cada jugador.
- appearances: información de goles, asistencias, tarjetas etc... de cada jugador en los diferentes partidos.

Para más información sobre las diferentes columnas de las tablas visite la pagina web mostrada.

#### Creación de la base de datos

In [434]:
#La base de datos se crea con el comando "CREATE DATABASE"
!beeline -u "jdbc:hive2://localhost:10000/" -e "CREATE DATABASE IF NOT EXISTS football COMMENT 'estadisticas de futbol' WITH DBPROPERTIES ('Creada por' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028130251_bcb6f910-6ac4-4f8c-82eb-e3f41563b8f7): CREATE DATABASE IF NOT EXISTS football COMMENT 'estadisticas de futbol' WITH DBPROPERTIES ('Creada por' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028130251_bcb6f910-6ac4-4f8c-82eb-e3f41563b8f7); Time taken: 0.249 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028130251_bcb6f910-6ac4-4f8c-82eb-e3f41563b8f7): CREATE DATABASE IF NOT EXISTS football COMMENT 'estadisticas de futbol' WITH DBPROPERTIES ('Creada por' = 'Sergio'

#### Tabla competitions

In [445]:
# Importamos los ficheros de datos a HDFS con el siguiente comando. COMPETITIONS
! hdfs dfs -put /media/notebooks/football/competitions.csv  /user/root

In [446]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE TABLE IF NOT EXISTS competitions (competition_id STRING, competition_code STRING, name STRING, sub_type STRING,type STRING, country_id INT, country_name STRING, domestic_league_code STRING, confederation STRING,url STRING) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028132609_6a974c1d-66f9-4552-82ca-b637b5c9afa2): CREATE TABLE IF NOT EXISTS competitions (competition_id STRING, competition_code STRING ,name STRING ,sub_type STRING ,type STRING, country_id INT, country_name STRING,domestic_league_code STRING,confederation STRING, url STRING)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028132609_6a974c1d-66f9-4552-82ca-b637b5c9afa2); Time taken: 0.42 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
IN

In [447]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
LOAD DATA INPATH '/user/root/competitions.csv' INTO TABLE competitions;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028132716_2fd772a3-a6c1-4919-8980-22f0b9f72247): LOAD DATA INPATH '/user/root/competitions.csv' INTO TABLE competitions
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028132716_2fd772a3-a6c1-4919-8980-22f0b9f72247); Time taken: 0.789 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028132716_2fd772a3-a6c1-4919-8980-22f0b9f72247): LOAD DATA INPATH '/user/root/competitions.csv' INTO TABLE competitions
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table football.competitions 

In [1]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT name, type, competition_id, country_id FROM competitions \
LIMIT 10;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028140938_1d192eed-cd85-42a0-a3a7-1fb7f108ac2f): SELECT name, type, competition_id, country_id FROM competitions  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:name, type:string, comment:null), FieldSchema(name:type, type:string, comment:null), FieldSchema(name:competition_id, type:string, comment:null), FieldSchema(name:country_id, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028140938_1d192eed-cd85-42a0-a3a7-1fb7f108ac2f); Time taken: 1.689 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_2023102814093

#### Tabla games

In [4]:
# Creación de la tabla
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE TABLE IF NOT EXISTS games (game_id INT, competition_id STRING, season INT, round STRING, date_ DATE, home_club_id INT, away_club_id INT, home_club_goals INT, away_club_goals INT, home_club_position INT, away_club_position INT, home_club_manager_name STRING, away_club_manager_name STRING, stadium STRING, attendance INT, referee STRING, url STRING, home_club_formation STRING, away_club_formation STRING, home_club_name STRING, away_club_name STRING, aggregate STRING, competition_type STRING) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028141356_77a02f92-89fd-488a-8c26-53b8e393e458): CREATE TABLE IF NOT EXISTS games (game_id INT, competition_id STRING, season INT, round STRING, date_ DATE, home_club_id INT, away_club_id INT, home_club_goals INT, away_club_goals INT, home_club_position INT, away_club_position INT, home_club_manager_name STRING, away_club_manager_name STRING, stadium STRING, attendance INT, referee STRING, url STRING, home_club_formation STRING, away_club_formation STRING, home_club_name STRING, away_club_name STRING, aggregate STRING, competition_type STRING)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)

In [5]:
# Importamos los ficheros de datos a HDFS con el siguiente comando. GAMES
! hdfs dfs -put /media/notebooks/football/games.csv  /user/root

In [7]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
LOAD DATA INPATH '/user/root/games.csv' INTO TABLE games;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028141542_ae36eb49-67a0-4e2f-ad47-28f7fb0fab5b): LOAD DATA INPATH '/user/root/games.csv' INTO TABLE games
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028141542_ae36eb49-67a0-4e2f-ad47-28f7fb0fab5b); Time taken: 0.055 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028141542_ae36eb49-67a0-4e2f-ad47-28f7fb0fab5b): LOAD DATA INPATH '/user/root/games.csv' INTO TABLE games
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table football.games from hdfs://namenode:8020/user/root

In [10]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT game_id, round, date_ FROM games \
LIMIT 10;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028141754_17eaf706-ef16-427d-926a-818bd6a5900e): SELECT game_id, round, date_ FROM games  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:game_id, type:int, comment:null), FieldSchema(name:round, type:string, comment:null), FieldSchema(name:date_, type:date, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028141754_17eaf706-ef16-427d-926a-818bd6a5900e); Time taken: 0.098 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028141754_17eaf706-ef16-427d-926a-818bd6a5900e): SELECT game_id, round, date_ FROM games  LIMIT 

#### Tabla clubs

In [14]:
# Creación de la tabla
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE TABLE IF NOT EXISTS clubs (club_id INT, club_code STRING, name STRING, domestic_competition_id STRING, total_market_value INT, squad_size INT, average_age FLOAT, foreigners_number INT, foreigners_percentage FLOAT, national_team_players INT, stadium_name STRING, stadium_seats INT, net_transfer_record STRING, coach_name STRING, last_season INT, url STRING) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028142744_c34b652c-fda5-459f-a160-964e8c763538): CREATE TABLE IF NOT EXISTS clubs (club_id INT, club_code STRING, name STRING, domestic_competition_id STRING, total_market_value INT, squad_size INT, average_age FLOAT, foreigners_number INT, foreigners_percentage FLOAT, national_team_players INT, stadium_name STRING, stadium_seats INT, net_transfer_record STRING, coach_name STRING, last_season INT, url STRING)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_2023102814274

In [15]:
# Importamos los ficheros de datos a HDFS con el siguiente comando. CLUBS
! hdfs dfs -put /media/notebooks/football/clubs.csv  /user/root

put: `/user/root/clubs.csv': File exists


In [16]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
LOAD DATA INPATH '/user/root/clubs.csv' INTO TABLE clubs;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028142756_943be3ba-6920-43e9-8875-f3963aeeedf9): LOAD DATA INPATH '/user/root/clubs.csv' INTO TABLE clubs
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028142756_943be3ba-6920-43e9-8875-f3963aeeedf9); Time taken: 0.045 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028142756_943be3ba-6920-43e9-8875-f3963aeeedf9): LOAD DATA INPATH '/user/root/clubs.csv' INTO TABLE clubs
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table football.clubs from hdfs://namenode:8020/user/root

In [18]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT club_id, name, total_market_value FROM clubs \
LIMIT 10;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028142905_892305cd-5d13-456d-b540-78ef2ec64dc3): SELECT club_id, name, total_market_value FROM clubs  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:club_id, type:int, comment:null), FieldSchema(name:name, type:string, comment:null), FieldSchema(name:total_market_value, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028142905_892305cd-5d13-456d-b540-78ef2ec64dc3); Time taken: 0.09 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028142905_892305cd-5d13-456d-b540-78ef2ec64dc3): SELECT club_id, name, tot

#### Tabla players

In [19]:
# Creación de la tabla
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE TABLE IF NOT EXISTS players (player_id INT, first_name STRING, last_name STRING, name STRING, last_season INT, current_club_id INT, player_code STRING, country_of_birth STRING, city_of_birth STRING, country_of_citizenship STRING , date_of_birth DATE, sub_position STRING, position STRING , foot STRING, height_in_cm INT, market_value_in_eur FLOAT, highest_market_value_in_eur FLOAT, contract_expiration_date DATE , agent_name STRING, image_url STRING, url STRING , current_club_domestic_competition_id STRING , current_club_name STRING) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028150130_41dfb5f2-3d9a-4981-a0b6-6187304dd638): CREATE TABLE IF NOT EXISTS players (player_id INT, first_name STRING, last_name STRING, name STRING, last_season INT, current_club_id INT, player_code STRING, country_of_birth STRING, city_of_birth STRING, country_of_citizenship STRING , date_of_birth DATE, sub_position STRING, position STRING , foot STRING, height_in_cm INT, market_value_in_eur FLOAT, highest_market_value_in_eur FLOAT, contract_expiration_date DATE , agent_name STRING, image_url STRING, url STRING , current_club_domestic_competition_id STRING , current_club_name STRING)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Se

In [20]:
# Importamos los ficheros de datos a HDFS con el siguiente comando. PLAYERS
! hdfs dfs -put /media/notebooks/football/players.csv  /user/root

In [21]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
LOAD DATA INPATH '/user/root/players.csv' INTO TABLE players;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028150504_e2eb5078-7adf-4118-830b-7c310b425461): LOAD DATA INPATH '/user/root/players.csv' INTO TABLE players
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028150504_e2eb5078-7adf-4118-830b-7c310b425461); Time taken: 0.029 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028150504_e2eb5078-7adf-4118-830b-7c310b425461): LOAD DATA INPATH '/user/root/players.csv' INTO TABLE players
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table football.players from hdfs://namenode:8020

In [22]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT last_season, current_club_id, date_of_birth FROM players \
LIMIT 10;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028150810_06abf212-05d6-4721-a159-0d166c6ad187): SELECT last_season, current_club_id, date_of_birth FROM players  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:last_season, type:int, comment:null), FieldSchema(name:current_club_id, type:int, comment:null), FieldSchema(name:date_of_birth, type:date, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028150810_06abf212-05d6-4721-a159-0d166c6ad187); Time taken: 0.095 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028150810_06abf212-05d6-4721-a159-0d166c6ad187): SELE

#### Tabla appearances

In [23]:
# Creación de la tabla
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE TABLE IF NOT EXISTS appearances (appearance_id STRING, game_id INT, player_id INT, player_club_id INT, player_current_club_id INT, date_ DATE , player_name STRING, competition_id STRING, yellow_cards INT, red_cards INT, goals INT, assists INT, minutes_played INT) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028151328_ec262bce-7f44-4488-bb5d-02e03596745b): CREATE TABLE IF NOT EXISTS appearances (appearance_id STRING, game_id INT, player_id INT, player_club_id INT, player_current_club_id INT, date_ DATE , player_name STRING, competition_id STRING, yellow_cards INT, red_cards INT, goals INT, assists INT, minutes_played INT)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028151328_ec262bce-7f44-4488-bb5d-02e03596745b); Time taken: 0.017 seconds
INFO  : Concurrency mode i

In [24]:
# Importamos los ficheros de datos a HDFS con el siguiente comando. APPEARANCES
! hdfs dfs -put /media/notebooks/football/appearances.csv  /user/root

In [25]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
LOAD DATA INPATH '/user/root/appearances.csv' INTO TABLE appearances;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028151522_ab674d52-9b82-4938-bdd4-d12e88a938a2): LOAD DATA INPATH '/user/root/appearances.csv' INTO TABLE appearances
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231028151522_ab674d52-9b82-4938-bdd4-d12e88a938a2); Time taken: 0.046 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028151522_ab674d52-9b82-4938-bdd4-d12e88a938a2): LOAD DATA INPATH '/user/root/appearances.csv' INTO TABLE appearances
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table football.appearances from 

In [26]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT goals, red_cards, minutes_played FROM appearances \
LIMIT 10;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231028151642_a4665f8b-bab6-435d-9847-b363bba0c084): SELECT goals, red_cards, minutes_played FROM appearances  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:goals, type:int, comment:null), FieldSchema(name:red_cards, type:int, comment:null), FieldSchema(name:minutes_played, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231028151642_a4665f8b-bab6-435d-9847-b363bba0c084); Time taken: 0.091 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231028151642_a4665f8b-bab6-435d-9847-b363bba0c084): SELECT goals, red_cards

#### Tabla club_games

In [7]:
# Creación de la tabla
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE TABLE IF NOT EXISTS club_games (game_id INT, club_id INT, own_goals INT, own_position INT, own_manager_name STRING, opponent_id INT, opponent_goals INT, opponent_position INT, opponent_manager_name STRING, hosting STRING, is_win INT) \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023');"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029090050_1c8fb689-c2ad-4da3-a20c-d273f0b32376): CREATE TABLE IF NOT EXISTS club_games (game_id INT, club_id INT, own_goals INT, own_position INT, own_manager_name STRING, opponent_id INT, opponent_goals INT, opponent_position INT, opponent_manager_name STRING, hosting STRING, is_win INT)  ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,'  TBLPROPERTIES ('Autor' = 'Sergio', 'Fecha' = '25/10/2023')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231029090050_1c8fb689-c2ad-4da3-a20c-d273f0b32376); Time taken: 0.016 seconds
INFO  : Concurrency mode is disabled, not creating a loc

In [8]:
# Importamos los ficheros de datos a HDFS con el siguiente comando. CLUB_GAMES
! hdfs dfs -put /media/notebooks/football/club_games.csv  /user/root

In [9]:
# Introduccion de los datos
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
LOAD DATA INPATH '/user/root/club_games.csv' INTO TABLE club_games;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029090217_66f1e19f-f960-46c0-b109-e6a3281d7c75): LOAD DATA INPATH '/user/root/club_games.csv' INTO TABLE club_games
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20231029090217_66f1e19f-f960-46c0-b109-e6a3281d7c75); Time taken: 0.027 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231029090217_66f1e19f-f960-46c0-b109-e6a3281d7c75): LOAD DATA INPATH '/user/root/club_games.csv' INTO TABLE club_games
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table football.club_games from hdfs:

In [12]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT game_id, club_id, is_win FROM club_games \
LIMIT 10;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029090405_49e330da-ff0a-49b7-b084-8e91d01913fb): SELECT game_id, club_id, is_win FROM club_games  LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:game_id, type:int, comment:null), FieldSchema(name:club_id, type:int, comment:null), FieldSchema(name:is_win, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20231029090405_49e330da-ff0a-49b7-b084-8e91d01913fb); Time taken: 0.067 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231029090405_49e330da-ff0a-49b7-b084-8e91d01913fb): SELECT game_id, club_id, is_win FROM clu

In [13]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "SHOW TABLES"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029090449_e787d08c-baf8-4451-b96a-ef89f5852270): SHOW TABLES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20231029090449_e787d08c-baf8-4451-b96a-ef89f5852270); Time taken: 0.013 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20231029090449_e787d08c-baf8-4451-b96a-ef89f5852270): SHOW TABLES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20231029090449_e787d08c-baf8-4451-b96a-ef89f5852270); Time

### Las consultas
#### 1- ¿Cuántos goles realizó cada club de la Liga española y en el año 2022?
Para contestar la pregunta, vamos a unir las tablas con INNER JOIN tal y como viene expuesto en la imagen de las diferentes tablas; seleccionamos en el nombre de la competición "laliga" y la temporada 2022, agrupamos por el nombre de los clubes y sumamos los goles realizados de cada uno de ellos:

In [36]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT clubs.name, SUM(appearances.goals) as goals FROM competitions \
INNER JOIN games \
ON (competitions.competition_id = games.competition_id AND competitions.name = 'laliga' AND games.season = 2022) \
INNER JOIN club_games \
ON (games.game_id = club_games.game_id) \
INNER JOIN clubs \
ON (club_games.club_id = clubs.club_id) \
INNER JOIN players \
ON (clubs.club_id = players.current_club_id) \
INNER JOIN appearances \
ON (games.game_id = appearances.game_id AND players.player_id = appearances.player_id)\
GROUP BY clubs.name;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231030083513_d49fd50f-9b66-41c4-98b5-167a69a9eaa9): SELECT clubs.name, SUM(appearances.goals) as goals FROM competitions  INNER JOIN games  ON (competitions.competition_id = games.competition_id AND competitions.name = 'laliga' AND games.season = 2022)  INNER JOIN club_games  ON (games.game_id = club_games.game_id)  INNER JOIN clubs  ON (club_games.club_id = clubs.club_id)  INNER JOIN players  ON (clubs.club_id = players.current_club_id)  INNER JOIN appearances  ON (games.game_id = appearances.game_id AND players.player_id = appearances.player_id) GROUP BY clubs.name
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : No Stats for football@competitions, Columns: competition_id, name
INFO  : No Stats for football@games, Columns: competition_id, seas

INFO  : Hadoop job information for Stage-4: number of mappers: 1; number of reducers: 1
INFO  : 2023-10-30 08:36:33,229 Stage-4 map = 0%,  reduce = 0%
INFO  : 2023-10-30 08:36:37,319 Stage-4 map = 100%,  reduce = 0%, Cumulative CPU 1.14 sec
INFO  : 2023-10-30 08:36:42,450 Stage-4 map = 100%,  reduce = 100%, Cumulative CPU 3.14 sec
INFO  : MapReduce Total cumulative CPU time: 3 seconds 140 msec
INFO  : Ended Job = job_1698494732979_0051
INFO  : MapReduce Jobs Launched: 
INFO  : Stage-Stage-16: Map: 1   Cumulative CPU: 14.23 sec   HDFS Read: 9936187 HDFS Write: 356418833 SUCCESS
INFO  : Stage-Stage-17: Map: 1   Cumulative CPU: 2.57 sec   HDFS Read: 16613913 HDFS Write: 8076 SUCCESS
INFO  : Stage-Stage-13: Map: 2   Cumulative CPU: 20.09 sec   HDFS Read: 356710333 HDFS Write: 2236152 SUCCESS
INFO  : Stage-Stage-11: Map: 1   Cumulative CPU: 4.2 sec   HDFS Read: 116706805 HDFS Write: 735 SUCCESS
INFO  : Stage-Stage-4: Map: 1  Reduce: 1   Cumulative CPU: 3.14 sec   HDFS Read: 8295 HDFS Write:

##### 2 - Mostrar a los diferentes jugadores con sus respectivos clubes y la cantidad de goles marcados por cada uno de ellos.

En primer lugar vamos a crear una *View*, ya que nos será util para la siguiente consulta. Al igual que antes seleccionamos la temporada 2022 y "laliga" agrupamos por clubes y nor el nombre de jugadores y sumamos la cantidad de goles que consigue cada uno: 

In [30]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
CREATE VIEW players_goals_liga AS \
SELECT players.name as name_players, clubs.name as name_clubs, SUM(appearances.goals) as goals FROM competitions \
INNER JOIN games \
ON (competitions.competition_id = games.competition_id AND competitions.name = 'laliga' AND games.season = 2022) \
INNER JOIN club_games \
ON (games.game_id = club_games.game_id) \
INNER JOIN clubs \
ON (club_games.club_id = clubs.club_id) \
INNER JOIN players \
ON (clubs.club_id = players.current_club_id) \
INNER JOIN appearances \
ON (games.game_id = appearances.game_id AND players.player_id = appearances.player_id)\
GROUP BY players.name, clubs.name;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029104858_80abbbeb-e7fd-4f6d-ac9a-840c24b9f7c6): CREATE VIEW players_goals_liga AS  SELECT players.name as name_players, clubs.name as name_clubs, SUM(appearances.goals) as goals FROM competitions  INNER JOIN games  ON (competitions.competition_id = games.competition_id AND competitions.name = 'laliga' AND games.season = 2022)  INNER JOIN club_games  ON (games.game_id = club_games.game_id)  INNER JOIN clubs  ON (club_games.club_id = clubs.club_id)  INNER JOIN players  ON (clubs.club_id = players.current_club_id)  INNER JOIN appearances  ON (games.game_id = appearances.game_id AND players.player_id = appearances.player_id) GROUP BY players.name, clubs.name
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : No Stats for football@competitions, Col

In [31]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT * FROM players_goals_liga;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029105006_bbe2c9d0-5f99-4be9-91d0-7cd57c961102): SELECT * FROM players_goals_liga
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : No Stats for football@competitions, Columns: competition_id, name
INFO  : No Stats for football@games, Columns: competition_id, season, game_id
INFO  : No Stats for football@club_games, Columns: club_id, game_id
INFO  : No Stats for football@clubs, Columns: club_id, name
INFO  : No Stats for football@players, Columns: player_id, current_club_id, name
INFO  : No Stats for football@appearances, Columns: player_id, game_id, goals
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:players_goals_liga.name_players, type:string, comment:null), FieldS

+----------------------------------+--------------------------------+---------------------------+
| players_goals_liga.name_players  | players_goals_liga.name_clubs  | players_goals_liga.goals  |
+----------------------------------+--------------------------------+---------------------------+
| Adu Ares                         | Athletic Bilbao                | 0                         |
| Aitor Paredes                    | Athletic Bilbao                | 0                         |
| Ander Capa                       | Athletic Bilbao                | 0                         |
| Ander Herrera                    | Athletic Bilbao                | 0                         |
| Asier Villalibre                 | Athletic Bilbao                | 0                         |
| Dani García                      | Athletic Bilbao                | 0                         |
| Dani Vivian                      | Athletic Bilbao                | 1                         |
| Gorka Guruzeta    

Beeline version 3.1.2 by Apache Hive
Closing: 0: jdbc:hive2://localhost:10000/football


##### 3 - Mostrar al máximo goleador de cada club junto con su equipo y la cantidad de goles.
Prara realizar esta consulta vamos a usar la View creada la consulta anterior. Vamos a realizar un INNER JOIN con de la View junto con el output de Agrupar por clubes y sus máximos goleadores, mostrando así el nombre de los jugadores que coincidan con el número de máximo de goles y su respectivo club:

In [35]:
!beeline -u "jdbc:hive2://localhost:10000/football" -e "\
SELECT players_goals_liga.name_players, players_goals_liga.name_clubs, players_goals_liga.goals FROM players_goals_liga \
JOIN (SELECT name_clubs, MAX(goals) AS max_goals FROM players_goals_liga \
GROUP BY name_clubs) max_goals_per_club \
ON players_goals_liga.name_clubs = max_goals_per_club.name_clubs AND players_goals_liga.goals = max_goals_per_club.max_goals;"

Connecting to jdbc:hive2://localhost:10000/football
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20231029113237_cff3e290-00af-4980-8126-49c420c9710c): SELECT players_goals_liga.name_players, players_goals_liga.name_clubs, players_goals_liga.goals FROM players_goals_liga  JOIN (SELECT name_clubs, MAX(goals) AS max_goals FROM players_goals_liga  GROUP BY name_clubs) max_goals_per_club  ON players_goals_liga.name_clubs = max_goals_per_club.name_clubs AND players_goals_liga.goals = max_goals_per_club.max_goals
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : No Stats for football@competitions, Columns: competition_id, name
INFO  : No Stats for football@games, Columns: competition_id, season, game_id
INFO  : No Stats for football@club_games, Columns: club_id, game_id
INFO  : No Stats for football@clubs, Columns: club_id, name
INFO  : No Stats for 

INFO  : 2023-10-29 11:34:27,864 Stage-26 map = 100%,  reduce = 0%, Cumulative CPU 27.84 sec
INFO  : MapReduce Total cumulative CPU time: 27 seconds 840 msec
INFO  : Ended Job = job_1698494732979_0040
INFO  : Launching Job 8 out of 21
INFO  : Starting task [Stage-34:MAPRED] in serial mode
INFO  : Number of reduce tasks is set to 0 since there's no reduce operator
INFO  : number of splits:2
INFO  : Submitting tokens for job: job_1698494732979_0041
INFO  : Executing with tokens: []
INFO  : The url to track the job: http://yarnmaster:8088/proxy/application_1698494732979_0041/
INFO  : Starting Job = job_1698494732979_0041, Tracking URL = http://yarnmaster:8088/proxy/application_1698494732979_0041/
INFO  : Kill Command = /app/hadoop-3.3.1/bin/mapred job  -kill job_1698494732979_0041
INFO  : Hadoop job information for Stage-34: number of mappers: 2; number of reducers: 0
INFO  : 2023-10-29 11:34:33,475 Stage-34 map = 0%,  reduce = 0%
INFO  : 2023-10-29 11:34:45,684 Stage-34 map = 50%,  reduce

Closing: 0: jdbc:hive2://localhost:10000/football
